In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Network Structure

In [2]:
class network_structure():
    def __init__(self, edges_nodes_matrix):
        # Structure of edges_nodes_matrix
        # An array of 2d-arrays, with each 2d-array encoding the start and end node indices of an edge.
        
        assert edges_nodes_matrix.shape[1] == 2, "We must have edges_nodes_matrix.shape[1] == 2"
        
        self.edges_nodes_matrix = edges_nodes_matrix
        self.nodes = np.unique(self.edges_nodes_matrix.flatten())
        self.edges = np.array(range(self.edges_nodes_matrix.shape[0]))
        self.num_nodes = len(self.nodes)
        self.num_edges = len(self.edges)
        
        self.construct_nodes_dict()
        self.construct_edges_dict()
#         self.construct_depth_dicts()
#         self.construct_height_dicts()
    
        origin_nodes = [node for node in self.nodes if self.nodes_dict[node]['incoming_edges'] == []]
        destination_nodes = [node for node in self.nodes if self.nodes_dict[node]['outgoing_edges'] == []]
        
        assert len(origin_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        assert len(destination_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        self.origin_node = origin_nodes[0]
        self.destination_node = destination_nodes[0]
        self.intermediate_nodes = [node for node in self.nodes if node not in [self.origin_node, self.destination_node]]
        
        self.start_edges = self.nodes_dict[self.origin_node]["outgoing_edges"]
        self.end_edges = self.nodes_dict[self.destination_node]["incoming_edges"]
        
        self.construct_routes_dict()
    
        return

    
    # Network structure functions:
    
    def construct_nodes_dict(self):
        self.nodes_dict = {}
        for node in self.nodes:
            self.nodes_dict[node] = {}
            self.nodes_dict[node]['incoming_edges'] = list((self.edges_nodes_matrix[:, 1] == node).nonzero()[0])
            self.nodes_dict[node]['outgoing_edges'] = list((self.edges_nodes_matrix[:, 0] == node).nonzero()[0])
            
            self.nodes_dict[node]['previous_nodes'] = list(set(self.edges_nodes_matrix[edge, 0] for edge in self.nodes_dict[node]['incoming_edges']))
            self.nodes_dict[node]['next_nodes'] = list(set(self.edges_nodes_matrix[edge, 1] for edge in self.nodes_dict[node]['outgoing_edges']))
        
        return

    def construct_edges_dict(self):
        self.edges_dict = {}

        for edge in self.edges:
            self.edges_dict[edge] = {}

        for node in self.nodes:
            if self.nodes_dict[node]['incoming_edges'] != []:
                for edge in self.nodes_dict[node]['incoming_edges']:
                    self.edges_dict[edge]['end_node'] = node

            if self.nodes_dict[node]['outgoing_edges'] != []:
                for edge in self.nodes_dict[node]['outgoing_edges']:
                    self.edges_dict[edge]['start_node'] = node

        for edge in self.edges:
            self.edges_dict[edge] = {}
            start_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['outgoing_edges']]))
            end_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['incoming_edges']]))
            self.edges_dict[edge]['start_node'] = start_node
            self.edges_dict[edge]['end_node'] = end_node
            self.edges_dict[edge]['incoming_edges'] = self.nodes_dict[start_node]['incoming_edges']
            self.edges_dict[edge]['outgoing_edges'] = self.nodes_dict[end_node]['outgoing_edges']
        return
    
    def construct_routes_dict(self):
        self.routes = [[edge] for edge in self.start_edges]
        
        route_construction_finished = False
        while route_construction_finished is False:
            route_construction_finished = True
            routes_temp = []
            for route_segment in self.routes:
                
#                 print()
#                 print("route_segment:", route_segment)
#                 print("route_segment[-1]:", route_segment[-1])
#                 print("self.edges_dict[route_segment[-1]]:", self.edges_dict[route_segment[-1]])
#                 print("self.edges_dict[route_segment[-1]][outgoing_edges]:", self.edges_dict[route_segment[-1]]["outgoing_edges"])
# #                 print()
                
                if self.edges_dict[route_segment[-1]]["outgoing_edges"] != []:
                    route_construction_finished = False
                    for edge in self.edges_dict[route_segment[-1]]["outgoing_edges"]:
                        routes_temp.append(route_segment + [edge])
                else:
                    routes_temp.append(route_segment)
                
#                 print("routes_temp:", routes_temp)
                
                
            if route_construction_finished is True:
                break
            self.routes = routes_temp
            
#             print()
#             print("self.routes:", self.routes)
#             print()

        return
    
    def print_nodes_dict(self):
        print()
        print("Printing nodes:")
        for node_key, node_value in self.nodes_dict.items():
            print()
            print("Node", node_key)
            for key, value in node_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_edges_dict(self):
        print()
        print("Printing edges:")
        for edge_key, edge_value in self.edges_dict.items():
            print()
            print("Edge", edge_key)
            for key, value in edge_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_routes(self):
        print()
        print("Printing routes:")
        print()
        for route_index, route_edges in enumerate(self.routes):
            print("Route", route_index, ":", route_edges)
        return
    
    def print_all(self):
        print()
        print("Printing network information:")
        print()
        print("Origin node:", self.origin_node)
        print("Destination node:", self.destination_node)
        print("Number of nodes:", self.num_nodes)
        print()
        print("Starting edges:", self.start_edges)
        print("Ending edges:", self.end_edges)
        print("Number of edges:", self.num_edges)
        
        self.print_nodes_dict()
        self.print_edges_dict()
        self.print_routes()
        
        return
    

In [3]:
# list_1 = [1, 2]
# list_2 = [3, 4, 5]
# list_1 = list_1 + [6]
# list_2.append(list_1)
# list_2

In [4]:
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
netw_example = network_structure(edges_nodes_matrix)

# print("netw_example.edges:", netw_example.edges)
# print()
# print("netw_example.nodes:", netw_example.nodes)
# print()
# print("netw_example.edges_dict:", netw_example.edges_dict)
# print()
# print("netw_example.nodes_dict:", netw_example.nodes_dict)
# print()
# print("netw_example.routes:", netw_example.routes)
# print()

# netw_example.print_nodes_dict()
# netw_example.print_edges_dict()
# netw_example.print_routes()

netw_example.print_all()
# netw_example.intermediate_nodes



Printing network information:

Origin node: 0
Destination node: 3
Number of nodes: 4

Starting edges: [0, 1]
Ending edges: [3, 4]
Number of edges: 5

Printing nodes:

Node 0
incoming_edges : []
outgoing_edges : [0, 1]
previous_nodes : []
next_nodes : [1, 2]

Node 1
incoming_edges : [0]
outgoing_edges : [2, 3]
previous_nodes : [0]
next_nodes : [2, 3]

Node 2
incoming_edges : [1, 2]
outgoing_edges : [4]
previous_nodes : [0, 1]
next_nodes : [3]

Node 3
incoming_edges : [3, 4]
outgoing_edges : []
previous_nodes : [1, 2]
next_nodes : []


Printing edges:

Edge 0
start_node : 0
end_node : 1
incoming_edges : []
outgoing_edges : [2, 3]

Edge 1
start_node : 0
end_node : 2
incoming_edges : []
outgoing_edges : [4]

Edge 2
start_node : 1
end_node : 2
incoming_edges : [0]
outgoing_edges : [4]

Edge 3
start_node : 1
end_node : 3
incoming_edges : [0]
outgoing_edges : []

Edge 4
start_node : 2
end_node : 3
incoming_edges : [1, 2]
outgoing_edges : []


Printing routes:

Route 0 : [0, 2, 4]
Route 1 : [

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [5]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [43]:
def welfare_obj(T, network, lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    ## Explanation:
    # y_el and y_in are dictionaries, each taking sets of indices as follows:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional"
    assert tau.shape[0] == num_edges, "toll vector first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros((num_edges, T))
    for e in range(num_edges):
        for t in range(T):
            x_ex[e, t] += sum(y_el[(g, e, k, t)] for g in range(num_el) for k in [0, 1])
            x_ex[e, t] += sum(y_in[(g, e, 0, t)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros((num_edges, T))
    for e in range(num_edges):
        for t in range(T):
            x_gp[(e, t)] += sum(y_el[(g, e, 2, t)] for g in range(num_el))
            x_gp[(e, t)] += sum(y_in[(g, e, 1, t)] for g in range(num_in))
    
    ell_ex = np.zeros((num_edges, T))
    ell_gp = np.zeros((num_edges, T))
    for e in range(num_edges):
        for t in range(T):
            ell_ex[e, t] = sum(a_ex[p, e] * (x_ex[e, t] ** p) for p in range(5))
            ell_gp[e, t] = sum(a_gp[p, e] * (x_gp[e, t] ** p) for p in range(5))
    
    obj_E = 0
    obj_I = 0
    obj_R = 0
    
    for e in range(num_edges):
        for t in range(T):
            obj_E += sum(tau[e, t] * y_el[(g, e, 1, t)] for g in range(num_el)) \
                        + sum(y_el[(g, e, k, t)] * v_E_array[g] * ell_ex[e, t] for g in range(num_el) for k in [0, 1]) \
                        + sum(y_el[(g, e, 2, t)] * v_E_array[g] * ell_gp[e, t] for g in range(num_el))
            obj_I += sum(tau[e, t] * y_in[(g, e, 0, t)] for g in range(num_in)) \
                        + sum(y_in[(g, e, 0, t)] * v_I_array[g] * ell_ex[e, t] for g in range(num_in)) \
                        + sum(y_in[(g, e, 1, t)] * v_I_array[g] * ell_gp[e, t] for g in range(num_in))
            obj_R += sum(tau[e, t] * y_el[(g, e, 1, t)] for g in range(num_el)) \
                        + sum(tau[e, t] * y_in[(g, e, 0, t)] for g in range(num_in))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare

# Latency, at a given time during the time horizon.

def latency_total(network, tau, v_I_array, v_E_array, y_el, y_in, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    ## Explanation:
    # y_el and y_in are dictionaries, each taking sets of indices as follows:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_gp[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_gp[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
    
    latency_val = 0.0
    
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = sum(a_ex[p, e] * (x_ex[e] ** p) for p in range(5))
        
        ell_gp[e] = sum(a_gp[p, e] * (x_gp[e] ** p) for p in range(5))
        
        latency_val += x_ex[e] * ell_ex[e] + x_gp[e] * ell_gp[e]
    
    return latency_val



In [44]:
def proj_tau_B_11(T, network, tau, B, tau_max = 1.0, B_max = 1.0):
    
    num_edges = len(network.edges)
    num_nodes = len(network.nodes)

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
    B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

    objective = cp.Minimize(func)
        
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max * np.ones((num_edges, T))]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= sum(tau_feas[e, t] for e in route for t in range(T)) \
                    for route in network.routes]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Chinmay's Algorithm:

In [45]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [48]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, network, tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    # Variables:
    y_el = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                for t in range(T):
                    y_el[(g, e, k, t)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                for t in range(T):
                    y_in[(g, e, k, t)] = cp.Variable(1)
    
    x_ex = {}
    x_gp = {}
    for e in network.edges:
        for t in range(T):
            x_ex[(e, t)] = cp.Variable(1)
            x_gp[(e, t)] = cp.Variable(1)
    
    # Objective:
    func = 0.0
    for e in network.edges:
        for t in range(T):
            func += 1/5 * a_ex[4, e] * cp.power(x_ex[(e, t)], 5)
            func += 1/4 * a_ex[3, e] * cp.power(x_ex[(e, t)], 4)
            func += 1/3 * a_ex[2, e] * cp.power(x_ex[(e, t)], 3)
            func += 1/2 * a_ex[1, e] * cp.power(x_ex[(e, t)], 2)
            func += a_ex[0, e] * x_ex[(e, t)]
            func += 1/5 * a_gp[4, e] * cp.power(x_gp[(e, t)], 5)
            func += 1/4 * a_gp[3, e] * cp.power(x_gp[(e, t)], 4)
            func += 1/3 * a_gp[2, e] * cp.power(x_gp[(e, ts)], 3)
            func += 1/2 * a_gp[1, e] * cp.power(x_gp[(e, t)], 2)
            func += a_gp[0, e] * x_gp[(e, t)]

            func += sum(tau[e, t] * y_el[(g, e, 1, t)] / v_E_array[g] for g in range(num_el))
            func += sum(tau[e, t] * y_in[(g, e, 0, t)] / v_I_array[g] for g in range(num_in))

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, e, k, t)] >= 0.0 for g in range(num_el) for e in network.edges \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y_in[(g, e, k, t)] >= 0.0 for g in range(num_in) for e in network.edges \
                    for k in [0, 1] for t in range(T)]
        
    constraints += [sum(y_el[(g, e, k, t)] for e in network.start_edges for k in [0, 1, 2]) == 1.0 \
                    for g in range(num_el) for t in range(T)]
    constraints += [sum(y_in[(g, e, k, t)] for e in network.start_edges for k in [0, 1]) == 1.0 \
                    for g in range(num_in) for t in range(T)]
    for node in network.intermediate_nodes:
        constraints += [sum(y_el[(g, e, k, t)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1, 2]) \
                        == sum(y_el[(g, e, k, t)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1, 2]) \
                        for g in range(num_el) for t in range(T)]
        constraints += [sum(y_in[(g, e, k, t)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1]) \
                        == sum(y_in[(g, e, k, t)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1]) \
                        for g in range(num_in) for t in range(T)]
    
    for e in network.edges:
        for t in range(T):
            constraints += [x_ex[(e, t)] == sum(y_el[(g, e, k, t)] for g in range(num_el) for k in [0, 1]) \
                                       + sum(y_in[(g, e, 0, t)] for g in range(num_in)) ]
            constraints += [x_gp[(e, t)] == sum(y_el[(g, e, 2, t)] for g in range(num_el)) \
                                       + sum(y_in[(g, e, 1, t)] for g in range(num_in)) ]
    
    constraints += [sum(y_el[(g, e, 0, t)] * tau[e, t] for e in network.edges for t in range(T)) \
                    <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                for t in range(T):
                    y_el_values[(g, e, k, t)] = max(y_el[(g, e, k, t)].value[0], 0.0)

    y_in_values = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                for t in range(T):
                    y_in_values[(g, e, k, t)] = max(y_in[(g, e, k, t)].value[0], 0.0)

    return y_el_values, y_in_values


In [49]:
time_1 = time.time()

# TODO: Edit below to incorporate time

T = 5

# edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3], [0, 3]])
network_Braess_augmented = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

network = network_Braess_augmented

tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
B = 0.8
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 10.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
flow_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
tau_max = sum(a[k] * (flow_max ** k) for k in range(5))
B_max = tau_max

d = num_edges * T + 1
num_iters = 1000
tau = np.zeros((network.num_edges, T, num_iters))
tau_perturbed = np.zeros((network.num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[:, :, 0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[:, :, 0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[:, :, 0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(network.num_edges * T + 1)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((network.num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B_11(T, network, tau_perturbed[:, :, i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[:, :, i]:", tau[:, :, i])
    print("B[i]:", B[i])
    print("tau_perturbed[:, :, i]:", tau_perturbed[:, :, i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(T, network, tau = tau[:, :, i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

#     print()
#     print("y_el_values:", y_el_values)
#     print("y_in_values:", y_in_values)
#     print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(T, network, tau = tau_perturbed[:, :, i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(T, network, lambda_E, lambda_R, lambda_I, tau = tau[:, :, i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(T, network, lambda_E, lambda_R, lambda_I, tau = tau_perturbed[:, :, i], \
                                    v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    welfare_list.append(welfare)
    
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print("tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((network.num_edges, T)) * (welfare_perturbed - welfare):\n", \
#     tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((network.num_edges, T)) * (welfare_perturbed - welfare))
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((network.num_edges, T)) * (welfare_perturbed - welfare)
    
#     print()
#     print("B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare):\n", \
#          B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare))
#     print("B[i]:", B[i])
#     print("eta[i]:", eta[i])
#     print("delta[i]:", delta[i])
#     print("w_i[-1]:", w_i[-1])
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print()
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    tau[:, :, i+1], B[i+1] = proj_tau_B_11(T, network, tau[:, :, i+1], B[i+1], tau_max, B_max)
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.linalg.norm(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i], axis = 0)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [-0.17270136  0.05758753  0.07264814 -0.25192779 -0.03386322  0.00512126
  0.02838645 -0.10730023 -0.34369112  0.28186156  0.01086508  0.1389394
  0.37834632 -0.13774102 -0.20045702  0.22561978  0.28553126 -0.13253627
  0.07891363  0.1181676   0.18713966  0.0200779  -0.14676272 -0.18816808
 -0.0991266   0.35615142  0.19218107 -0.13479493  0.06238188  0.00917259
  0.08377547]
tau[:, :, i]: [[907.2 907.2 907.2 907.2 907.2]
 [907.2 907.2 907.2 907.2 907.2]
 [907.2 907.2 907.2 907.2 907.2]
 [907.2 907.2 907.2 907.2 907.2]
 [907.2 907.2 907.2 907.2 907.2]
 [907.2 907.2 907.2 907.2 907.2]]
B[i]: 388.8
tau_perturbed[:, :, i]: [[906.8336 907.3222 907.3541 906.6656 907.1282]
 [907.2109 907.2602 906.9724 906.4709 907.7979]
 [907.223  907.4947 908.0026 906.9078 906.7748]
 [907.6786 907.8057 906.9188 907.3674 907.4507]
 [907.597  907.2426 906.8887 906.8008 906.9897]
 [907.9555 907.6077 906.9141 907.3323 907.2195]]
B_perturbed[i]: 388.9777

Iter: 1
w_i: [ 0.03720756  0.12635706 -0.056

tau_diffs: [[0.00694838 0.05059545 0.0160941  0.04641756]
 [0.00807403 0.04299384 0.0269557  0.06244045]
 [0.00990252 0.02715456 0.03832741 0.04294182]
 [0.00638279 0.05530045 0.02731318 0.03768926]
 [0.00649538 0.05118789 0.03835492 0.05258688]]
B_diffs: [ 0.0019 -0.0057 -0.0269 -0.0028]

Iter: 8
w_i: [ 0.03168151  0.01594698 -0.02189372  0.14289559 -0.02250879  0.08618796
  0.03058436  0.09069542  0.07270317 -0.38949735  0.07514233 -0.42997432
 -0.15208134 -0.01092839  0.26968708  0.11094752  0.04957256  0.2481941
  0.15940603 -0.14534949  0.16550715  0.07171875 -0.17364421  0.13450787
  0.11639808 -0.23768879  0.109411    0.14971731  0.00808627  0.04417059
 -0.45510245]
tau[:, :, i]: [[907.1884 907.1934 907.2027 907.0665 907.0993]
 [907.2041 907.2524 907.2163 907.2136 907.3121]
 [907.1664 907.1083 907.2478 907.1501 907.1401]
 [907.1326 907.2125 907.1904 907.2681 907.2195]
 [907.2001 907.248  907.1759 907.2438 907.1826]
 [907.2629 907.2102 907.1469 907.1693 907.1399]]
B[i]: 388.8325


tau_diffs: [[0.01439132 0.02055116 0.00693614 0.01600094]
 [0.02714332 0.01901631 0.00457165 0.01046136]
 [0.02112818 0.02377898 0.00705833 0.01514464]
 [0.02212871 0.03081055 0.00827466 0.01711578]
 [0.0264318  0.02535074 0.00950999 0.02105255]]
B_diffs: [-9.10e-03 -1.90e-02 -1.00e-04 -1.21e-02]

Iter: 15
w_i: [-0.01946019  0.04594449 -0.02320083 -0.10224056 -0.22378582  0.20432909
 -0.29342818  0.01422746 -0.07774432  0.24783307 -0.22032322  0.22173093
 -0.43739081  0.19190068  0.03456357 -0.15958956 -0.05231045 -0.00990127
  0.34293597  0.01231092 -0.1334309  -0.24853301 -0.10132159 -0.15373095
  0.30870006 -0.04152948 -0.01600487  0.02609446  0.13693378 -0.18728566
  0.05066111]
tau[:, :, i]: [[907.1579 907.173  907.1974 907.011  907.1154]
 [907.1906 907.2473 907.163  907.1735 907.364 ]
 [907.1577 907.1771 907.3009 907.1788 907.1158]
 [907.109  907.1717 907.151  907.2496 907.2248]
 [907.1698 907.2373 907.2006 907.2374 907.1494]
 [907.2572 907.1916 907.1047 907.1982 907.1204]]
B[i]:

tau_diffs: [[0.00568243 0.01322951 0.03649356 0.00534322]
 [0.01317042 0.01110225 0.02233182 0.00405956]
 [0.00894148 0.01266175 0.04641508 0.00497695]
 [0.00557494 0.01151955 0.03431997 0.00518748]
 [0.00580689 0.0088017  0.02201091 0.00765114]]
B_diffs: [-0.0039 -0.0066  0.0072  0.0008]

Iter: 22
w_i: [-0.24251164 -0.03730248  0.01777384 -0.04832789  0.03974578  0.03769026
  0.43934539 -0.00172214  0.1787016   0.27659994 -0.25736714 -0.07606239
  0.16183912  0.29080423 -0.10052449  0.08491512 -0.02783815 -0.17829113
  0.1995961   0.24077349  0.12435483 -0.29100966  0.17750505 -0.23064716
  0.07257816 -0.20644409 -0.00621536 -0.19812825 -0.09618725 -0.0443201
  0.13766312]
tau[:, :, i]: [[907.16   907.1882 907.1925 907.0328 907.1142]
 [907.1869 907.2375 907.1821 907.1682 907.3894]
 [907.17   907.171  907.3264 907.1831 907.0957]
 [907.0983 907.1727 907.0922 907.2755 907.2069]
 [907.1484 907.2315 907.2203 907.2284 907.1529]
 [907.2857 907.2032 907.1017 907.2221 907.1059]]
B[i]: 388.974


tau_diffs: [[0.03840195 0.00336303 0.01542466 0.00554797]
 [0.02542892 0.00561694 0.01894941 0.00504083]
 [0.02682536 0.00418569 0.03362796 0.00679485]
 [0.01414638 0.00507149 0.02559727 0.0084788 ]
 [0.02940748 0.00629603 0.05534925 0.00621289]]
B_diffs: [-0.0186  0.0005 -0.0005  0.0038]

Iter: 29
w_i: [ 0.25272544 -0.27497694  0.08575606 -0.20506511  0.22377779  0.05304201
 -0.15679942  0.36175342  0.08838819 -0.02772123  0.03261444 -0.14640802
 -0.14353314  0.12140168 -0.11232979 -0.33318284  0.09053794 -0.20204311
  0.15141155  0.09899978  0.29631405  0.0413764   0.13112321 -0.05351058
  0.24298008 -0.11471174 -0.01783573 -0.30313784  0.03200629  0.13571172
 -0.19379068]
tau[:, :, i]: [[907.1794 907.1883 907.2033 907.0455 907.0951]
 [907.2016 907.2121 907.2241 907.1544 907.3959]
 [907.1648 907.1795 907.3294 907.196  907.1091]
 [907.0835 907.1899 907.1004 907.273  907.2401]
 [907.1013 907.2265 907.257  907.2122 907.1802]
 [907.2978 907.2108 907.1034 907.2578 907.0733]]
B[i]: 389.006

tau_diffs: [[0.01226907 0.0044643  0.00120416 0.00659318]
 [0.01661776 0.00411947 0.00410609 0.01396925]
 [0.02252266 0.00502195 0.0021166  0.01020441]
 [0.01431468 0.00673721 0.00262869 0.01246074]
 [0.01664962 0.00611964 0.00139642 0.00627774]]
B_diffs: [ 9.9e-03  1.0e-04 -9.0e-04  0.0e+00]

Iter: 36
w_i: [ 0.08445487  0.24020013 -0.17704224  0.04901386 -0.02666695 -0.29638456
  0.09517291  0.05813957  0.04228936 -0.03858727 -0.11247909  0.01842768
  0.2898681   0.02417776  0.05354292  0.15197516  0.18287128  0.11039104
 -0.12592617  0.41646039 -0.10625438  0.40109576  0.17806155  0.06823232
  0.07913823  0.3951012  -0.25013035 -0.05819893  0.02367824 -0.0344833
 -0.00825036]
tau[:, :, i]: [[907.1781 907.2061 907.1989 907.0553 907.0901]
 [907.1999 907.2037 907.2069 907.1456 907.378 ]
 [907.1594 907.2009 907.3163 907.1908 907.1049]
 [907.0915 907.1954 907.095  907.2535 907.2421]
 [907.0956 907.236  907.256  907.1977 907.1838]
 [907.2889 907.2362 907.0952 907.258  907.0633]]
B[i]: 389.

tau_diffs: [[0.03031831 0.01470748 0.00997296 0.01423095]
 [0.03029521 0.00726911 0.01510397 0.011063  ]
 [0.0312642  0.01850135 0.00679559 0.01336413]
 [0.03321506 0.02246953 0.00926499 0.01251639]
 [0.04627937 0.01334841 0.00589237 0.01246635]]
B_diffs: [-0.0198 -0.007  -0.006  -0.0061]

Iter: 43
w_i: [-0.24712845  0.05639854 -0.13411802  0.00047012  0.09646938 -0.22071286
 -0.25530331  0.03247955 -0.34737663  0.12640551 -0.16778739 -0.36791724
 -0.04289762  0.03792049 -0.15014668 -0.17140115 -0.27942573  0.10144977
  0.01969837 -0.12350642 -0.19082917 -0.03380605 -0.40272873 -0.06631068
  0.14889988  0.06668048 -0.07005746  0.18988224  0.01650293  0.09376401
 -0.23094353]
tau[:, :, i]: [[907.195  907.2233 907.2054 907.0469 907.0634]
 [907.1589 907.1724 907.1876 907.1541 907.373 ]
 [907.1898 907.2161 907.3414 907.2036 907.1212]
 [907.1026 907.2285 907.1322 907.2394 907.2674]
 [907.1234 907.2825 907.29   907.2047 907.2174]
 [907.3318 907.2246 907.1094 907.2948 907.0206]]
B[i]: 389.045

tau_diffs: [[0.00326497 0.00213307 0.05760729 0.0159778 ]
 [0.0049051  0.00315278 0.08426067 0.01313354]
 [0.00635925 0.00222711 0.09383374 0.0147594 ]
 [0.0050705  0.00225167 0.08642135 0.02037008]
 [0.00354683 0.00165529 0.20750284 0.0177519 ]]
B_diffs: [ 0.0002  0.001   0.0044 -0.014 ]

Iter: 50
w_i: [ 1.29755727e-01  2.67926636e-01  2.21895067e-01  9.27976882e-02
  4.75475368e-01  1.11616809e-01 -1.50543288e-01  1.84296107e-01
  1.80452578e-01  2.64117929e-01  1.52952169e-01  6.61509437e-02
 -8.65179123e-02 -2.34368455e-01  3.71656422e-02  2.73406273e-02
 -1.36361208e-02 -1.66978007e-02 -1.14631230e-01 -6.88081690e-02
  1.14116944e-01 -2.50894104e-04  6.75613498e-02 -7.30295795e-02
 -1.89660824e-01  1.51188727e-01 -1.86247817e-01 -8.77523192e-02
 -1.76355378e-01  2.07822213e-01  3.83356898e-01]
tau[:, :, i]: [[907.1924 907.1349 907.1838 907.085  907.1345]
 [907.1454 907.1404 907.1749 907.2215 907.4779]
 [907.1939 907.193  907.4477 907.229  907.1837]
 [907.1696 907.2385 907.1783 907

tau_diffs: [[0.00410366 0.01117229 0.04326211 0.00711126]
 [0.0060208  0.00786829 0.05377323 0.01220942]
 [0.00645678 0.01009455 0.0422734  0.00940798]
 [0.00831565 0.00757628 0.0501737  0.01127253]
 [0.01101499 0.00961197 0.0351346  0.01698823]]
B_diffs: [ 0.0017 -0.0023  0.0064  0.0019]

Iter: 57
w_i: [ 0.08941577  0.2289416  -0.05155494 -0.00515455 -0.30031318 -0.17136179
 -0.04636134  0.06176333  0.04100969  0.01176929 -0.25379094  0.01581985
  0.10757257  0.26367681  0.44856013  0.13724432 -0.01516453 -0.13724884
 -0.13740398  0.1309241   0.25490284 -0.27671959 -0.03611225  0.15312677
 -0.18354336  0.14653603  0.13851073 -0.3452065   0.07284887  0.15334137
 -0.00675812]
tau[:, :, i]: [[907.1679 907.1315 907.2223 907.0566 907.1497]
 [907.1126 907.1295 907.167  907.1787 907.4763]
 [907.166  907.2081 907.4447 907.2359 907.186 ]
 [907.1823 907.2369 907.1605 907.2855 907.3272]
 [907.1524 907.2825 907.316  907.1685 907.0995]
 [907.3148 907.2912 907.0964 907.3399 906.9711]]
B[i]: 389.044

KeyboardInterrupt: 

In [34]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

IndexError: list index out of range

In [ ]:
y_el_values

In [13]:
y_in_values

{(0, 0, 0): 0.5821719216319861,
 (0, 0, 1): 0.2094961277106064,
 (0, 1, 0): 0.004180742490241613,
 (0, 1, 1): 0.20415120816716587,
 (0, 2, 0): 4.144949098608525e-09,
 (0, 2, 1): 1.1425231528975375e-07,
 (0, 3, 0): 0.5846215247840308,
 (0, 3, 1): 0.20704640616129735,
 (0, 4, 0): 5.016818349357077e-07,
 (0, 4, 1): 0.20833156737283695,
 (1, 0, 0): 0.7108227707122439,
 (1, 0, 1): 4.161077085583072e-09,
 (1, 1, 0): 0.28917722088739817,
 (1, 1, 1): 4.239280934608319e-09,
 (1, 2, 0): 5.664237206604805e-09,
 (1, 2, 1): 1.505274957584457e-07,
 (1, 3, 0): 0.7108226144311154,
 (1, 3, 1): 4.250472527391911e-09,
 (1, 4, 0): 0.28917737714979724,
 (1, 4, 1): 4.168614768110095e-09,
 (2, 0, 0): 1.5326673968831134e-06,
 (2, 0, 1): 1.8587508398815776e-09,
 (2, 1, 0): 0.9999984635749538,
 (2, 1, 1): 1.8988984661208134e-09,
 (2, 2, 0): 7.3891652192127915e-09,
 (2, 2, 1): 2.2095957174396205e-07,
 (2, 3, 0): 1.3042789283025461e-06,
 (2, 3, 1): 1.89848245727403e-09,
 (2, 4, 0): 0.9999986919690309,
 (2, 4, 1):

## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Unused Functions for Network:

In [ ]:

def construct_depth_dicts(self):
    self.depth_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['incoming_edges']) == 0]
    self.depth_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['incoming_edges']).issubset(set(edges_accounted))]
        self.depth_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.depths = list(self.depth_edges_dict.keys())

    self.depth_nodes_dict = {}
    for depth in self.depths:
        nodes_temp = np.array([self.edges_dict[edge]['start_node'] for edge in self.depth_edges_dict[depth]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.depth_nodes_dict[depth] = nodes_temp

    return

def construct_height_dicts(self):
    self.height_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['outgoing_edges']) == 0]
    self.height_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['outgoing_edges']).issubset(set(edges_accounted))]
        self.height_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.heights = list(self.height_edges_dict.keys())

    self.height_nodes_dict = {}
    for height in self.heights:
        nodes_temp = np.array([self.edges_dict[edge]['end_node'] for edge in self.height_edges_dict[height]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.height_nodes_dict[height] = nodes_temp

    return

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
